Group Name: AG 03.

Student Name (Student ID):

1. xxxx xxxxx (xxxxxxx)

2. xxxx xxxxx (xxxxxxx)

3. xxxx xxxxx (xxxxxxx)

# Question 2

## Introduction to question 2

In the second question of this assignment, we will explore the use of local search in genome assembly.

We will use local search to assemble (construct) a large part of the nucleotide sequence of the monkeypox virus, which has been downloaded from the National Center for Biotechnology Information in the United States. Please note that no additional or specialized knowledge of biology or bioinformatics is required for this assignment. (Actually, the technical specifics of bioinformatics have been adapted and simplified for the purposes of this computer science assignment, so if you are a biologist, please do not apply preexisting knowledge to solve the problem. Furthermore, you should not attempt to search up the genome on genomic databases to "guess" the actual sequence, since we are more interested in your coding methodology rather than your attempts at reproducing a known sequence.)

This is an introductory computer science assignment and not a bioinformatics assignment; we are simply using bioinformatics as a use case to illustrate the applicability of local search to the natural sciences. Therefore, no knowledge of bioinformatics is assumed or required. In the paragraphs that follow, I will give a short crash course which will cover all the domain knowledge you will need to know in order to tackle this problem.  

For technical reasons, when we analyze the nucleotide sequence (genome) of a virus, we usually cannot “read” it in one fell swoop. We have to read the genome in parts, because the genome is usually too long for the machine to read in a single sitting. To simplify things, a “read” is a single view of part of the genome; think of it as a SUBSTRING, a partial view of the whole genome. After we have generated multiple reads of a genome, we then have to “stitch”, or combine, the different reads of the genome together. This process of stitching up reads of a genome into the final sequence is known as genome assembly. However, the different reads of the genome cannot just be concatenated like usual string concatenation. It’s not a situation where you have one read, “Hello”, and another read, “World”, and all you need to do is concatenate both strings together to make “Hello World”. Among other reasons, there are two major reasons why you can’t do so:

1. You do not know which read came first. The reads are not ordered. How do you know “Hello” came after “World”? The answer is that you don’t. Imagine how complicated this situation might be if you had more than two reads. (This is indeed our situation, where we have $n$ reads, and $n>>2$.)

2. One read may contain a substring contained in another read. Specifically, without loss of generality, part of the ending $x$ characters of a read (i.e., suffix) might also be found in the starting $x$ positions (i.e., prefix) of another read.

- A computer scientist usually creates opportunities from problems. While this may be a “problem” in that you just can’t concatenate two strings blindly, the fact that strings contain shared “substrings” is actually a very helpful clue that you can use to “join” strings together. 

- Note that the choice of the value of $x$ could be a hyperparameter decided by the computer scientist.

## Your tasks

In this part of the assignment, you will work with (simulated) reads that I have generated from the nucleotide sequence of the monkeypox virus. In reality, bioinformatics is far more complicated, but here we will work with a simplified situation. Your task is to examine the reads that I have provided for you, and from there “infer” the nucleotide sequence that might have produced those reads. 

The reads are provided in the csv file `data.csv` which simply provides a list of unique strings. Note that you should NOT assume any particular ordering of the strings in this dataframe. In fact, the strings have already been shuffled randomly. 

NOTE: You are not allowed to use `pandas` or any other libraries apart from the Python STL to load the csv file.

### Task A (3 marks): 

Create a directed graph. The nodes in the graph are the strings in the list of reads. An edge should be drawn FROM read A TO read B if and only if a suffix (of length $x$) of read A is also a prefix (obviously, also of length $x$) of read B. For the purposes of the assignment, limit the value of $x$ to between 5 and 30, both inclusive. That is, to be clear, $5\leq x\leq 30$. The weight of an edge between read A and read B should be the NEGATED value of $x$, i.e. $-x$. 

In your Jupyter notebook, please report the number of edges in your graph. Provide a barplot or histogram which shows the number of edges with different weights or weight categories. In this task, you are free to use plotting libraries such as `matplotlib` or `seaborn` to plot this graph.

As an example, if read A is "TACTAGT" and read B is "TAGTCCCCT", then an edge is drawn FROM read A TO read B (i.e., $A \rightarrow B$) with weight of $-4$. This is because the 4-suffix "TAGT" is also the 4-prefix of read B; in other words, the last 4 characters of read A (a substring of length 4) overlap with the first 4 characters of read B (a substring of length 4).

### Task B (7 marks): 

From Task A, you now have a graph which shows connections between reads based on how they overlap, in theory you could draw a path through the graph and thereby derive the full sequence (genome).

Task B asks you to use local search method(s) to determine a path through this directed graph of strings. 

- You are expected to use simulated annealing and tune the relevant configuration settings and hyperparameters. The minimum requirement is to implement simulated annealing.

- Explain tha rationale behind the choice of scheduling strategy and parameters.

- However, you may also explore other search methods in addition to simulated annealing. Marks will be awarded for effort.

Note the following constraints:

1. The path has to go through each and every vertex exactly once. For computer scientists, this constraint is reminiscent of the "Traveling Salesman's Problem", except that unlike TSP, we should not need to go back to the starting vertex again. 

2. For the purposes of neighbor generation / action selection at each node, bear in mind that a path through the graph which minimizes the total number of nucleotides in the assembled sequence is the preferred path. To state that another way, the assembled sequence should be derived from a path that goes through EACH and EVERY vertex exactly once, however we want this assembled sequence to be AS SHORT AS POSSIBLE.

3. You are not given the starting (source/origin) or ending (destination) vertex.

4. For avoidance of ambiguity, no cycles are allowed. You must not visit a vertex more than once.

5. You are not allowed to use any libraries apart from the Python Standard Library.
No import statements which import libraries outside of the Python STL should be found within your answer for Task B.

Please remember to report the assembled sequence that you obtain. Although it would be great if you can come up with a good sequence, please feel reassured that we are more interested in your APPROACH to the problem, and so you can potentially get a reasonable score on this task even if your solution is "wrong". It is the process, rather than the result, which matters more.

In [72]:
# Problem Class
class Problem:
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When you create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)

In [84]:
# Use the following Node class to generate search tree
import math
class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost 

    #Extensions to Node Class
    def expand(problem, node):
        "Expand a node, generating the children nodes."
        s = node.state
        for action in problem.actions(s):
            s1 = problem.result(s, action)
            cost = node.path_cost + problem.action_cost(s, action, s1)
            return Node(s1, node, action, cost)
            

    def path_actions(node):
        "The sequence of actions to get to this node."
        if node.parent is None:
            return []  
        return path_actions(node.parent) + [node.action]


    def path_states(node):
        "The sequence of states to get to this node."
        if node in (cutoff, failure, None): 
            return []
        return path_states(node.parent) + [node.state]

In [74]:
# Code to generate neighbours, value of states, etc.
class TSP(Problem):
    #Implement TSP class here
    pass

## Part 1 - Implementation of a graph structure of data.csv. 


Input Data: to be structured into a dictionary using {index:read_sequence} 

### Graph Datatype
Directed Graph: to be built as an adjacency matrix

*Example of Directed Graph Data Format*
```
Adjacency Dictionary 
directed_graph = {
                    A : [(B,4), (C,1), ..., (F,3)], 
                    B : [(E,5), (H,3), ..., (J,3)], 
                    ...
                    }  #letters to be index and numbers to be overlap. 
```

```
Adjacency List 
directed_graph = [
                    [A, B, 5], 
                    [A, C, 3], 
                    [D, E, 2],
                    ....
                    ]  #letters to be index. 
```


For simplicity, decision to build graph as an adjacency list first. To build the graph, we will iterate through edge weights and count the numbers. This to be done as a once off, static output. For future optimization, this could be implemented as a lookup table of suffixes and prefixes in the format {suffix/prefix: id1, id2, idx} and set overlaps used to generate dynamic outputs. 

In [75]:
import csv
from pprint import pprint

# Import the data, discarding column 2 because its similar to column 1
data_reads = [] 
with open ('data.csv', 'r') as csv_file:
    for line in csv.reader(csv_file, delimiter=','):
        data_reads.append([line[0],line[2]])
#print (data)

data_reads = [[int(row[0]), row[1]] for row in data_reads[1:]] #discard the header row, convert types
pprint (data_reads[:5]) #🔴 since starts from zero, this can be a 1dimensional list accessed by index


[[0,
  'CTTGAATTGGTTCCTGGTATCATTAGGATCTCTGTCTCTCAACATCTGTTTAAGTTCATCGAGAACCACCTCCTCATTTTCCAGATAGTCAAACATTTTGACTGAATAGAAGTGAATGAGCTACTGTGAACTCTATACACCCGCACAACTAATGTCATTAAATATCATTTTTGAATGTATTTATACCATGTCAAAAACTTGTACAATTATTAATAAAAATAATTAGTGTTTAAATTTTACCAGTTCCAGATTTTACACCTCCGTTAACACCTCCATTAACCCCACTTTTTACACCACTGGACGATCCTCCTCCCCACATTCCACTGCCACTAGATGTATAAGTTTTAGATCCTTTATTACTACCATCATGTCCATGGATAAAGACACTCCACATGCCGCCACTACTACCCCCT'],
 [1,
  'ATCTTTAACGAACATATACCTAGATGGTTATTTACTAACAGACATTTTTTCAAGATCTATTGACAATAACTCCTATAGTTTCCACATCAACCAAGTAATGATCATCTATTGTTATATAACAATAACATAACTCTTTTCCATTTTTATCAGTATCTATATCAACGTCGTTGTAGTGAATAGTAGTCATTGATCTATTATATGAAACGGATATGTCTAGTTAATATTTTCTTTGATTTAAAGTCTATAGTCTTTACAAACATAATATCCTTATCCGACTTTATATTTCCTGTAGGGTGGCATAATTTTATTCTGCCTCCACAATCAGTGTTTCCAAATATATTACTAGACAATATTCCATATAGT'],
 [2,
  'TTGTACATGTAATGATTTAAAATGTGTAGTCATGCTTATTGATAAAGATCTAAAAATTAAAGCGGGTCCTCGGTACGTGCTTAACGCTATTAGTCCTCATGCCTATGATGTTTTTAGAAAATCTAATAACTTGAAAGAGATAATAGAAAATGCAGCTAAACAAAATCTAGACTCTATATCTATTTCTGTTAT

In [94]:
# Find the overlapping prefix and suffixes 
def overlap(readline: list, readlist: list, graph_output: list = None) -> str: # takes
    '''
    Lists should be in format [id, string]
    Takes a readline suffix and compares it against a readlist prefix for overlap in range 5 to 30
    output: 
    - matches are it appends it to the graph_ouput list 
    - format: [readline ID, readlist item ID, weight] i.e. A -> B, Weight 
    '''
    for i in range (5, 31): #🔴 (1) double check if range and slices are correct. (2), this can be made more efficient: do i in reverse and add an idx_matched as wel 
        
        for line in readlist: 
            suffix = readline[1][-i:]
            prefix = line[1][:i]
            #print (f'{suffix}, {prefix}')
            if suffix == prefix:
                #print (f'match! {prefix} : {line[0]}-{suffix}')
                graph_output.append([readline[0], line[0], i * -1])
                #graph_adj_list.append([print ('match!')

graph_adj_list = [] 
for i in range(len(data_reads)):
    overlap(data_reads[i], data_reads, graph_adj_list)

pprint (graph_adj_list[:10])

[[0, 302, -5],
 [0, 224, -16],
 [1, 511, -5],
 [1, 344, -6],
 [1, 228, -18],
 [2, 223, -22],
 [3, 548, -5],
 [3, 355, -24],
 [4, 436, -17],
 [5, 102, -5]]


In [95]:
# Charting
import plotly.express as px #using plotly because its prettier, easier and more extensible than seaborn. :D 
#🔴 Change to seaborn 
from collections import Counter

histogram = dict(Counter([line[2] for line in graph_adj_list]))
histogram = {'weights': list(histogram.keys()), 'count': list(histogram.values()) } #ref: https://stackoverflow.com/questions/62660718/how-to-create-a-bar-graph-using-plotly-express-from-a-dictionary

fig = px.bar(histogram, x = 'weights', y = 'count')
fig.show() 

## Understanding Problem and Node

# Graph for genome relationships

Construct a Graph class for the genomic relationships.

*Note: by design, the nodes are identified by Key, not actual genome portion. This is for readability.*

In [78]:
class Graph:
    """
    Taken from AIME4e, search.py 
    Modified to include __repr__. 

    A graph connects nodes (vertices) by edges (links). Each edge can also
    have a length associated with it. 
    The constructor call is something like:
        g = Graph({'A': {'B': 1, 'C': 2})
    this makes a graph with 3 nodes, A, B, and C, with an edge of length 1 from
    A to B,  and an edge of length 2 from A to C. You can also do:
        g = Graph({'A': {'B': 1, 'C': 2}, directed=False)
    This makes an undirected graph, so inverse links are also added. The graph
    stays undirected; if you add more links with g.connect('B', 'C', 3), then
    inverse link is also added. You can use g.nodes() to get a list of nodes,
    g.get('A') to get a dict of links out of A, and g.get('A', 'B') to get the
    length of the link from A to B. 'Lengths' can actually be any object at
    all, and nodes can be any hashable object."""

    def __init__(self, graph_dict=None, directed=True):
        self.graph_dict = graph_dict or {}
        self.directed = directed
        if not directed:
            self.make_undirected()

    def make_undirected(self):
        """Make a digraph into an undirected graph by adding symmetric edges."""
        for a in list(self.graph_dict.keys()):
            for (b, dist) in self.graph_dict[a].items():
                self.connect1(b, a, dist)

    def connect(self, A, B, distance=1):
        """Add a link from A and B of given distance, and also add the inverse
        link if the graph is undirected."""
        self.connect1(A, B, distance)
        if not self.directed:
            self.connect1(B, A, distance)

    def connect1(self, A, B, distance):
        """Add a link from A to B of given distance, in one direction only."""
        self.graph_dict.setdefault(A, {})[B] = distance #🔴 What is setdefault? 

    def get(self, a, b=None):
        """Return a link distance or a dict of {node: distance} entries.
        .get(a,b) returns the distance or None;
        .get(a) returns a dict of {node: distance} entries, possibly {}."""
        links = self.graph_dict.setdefault(a, {})
        if b is None:
            return links
        else:
            return links.get(b)

    def nodes(self):
        """Return a list of nodes in the graph."""
        s1 = set([k for k in self.graph_dict.keys()])
        s2 = set([k2 for v in self.graph_dict.values() for k2, v2 in v.items()])
        nodes = s1.union(s2)
        return list(nodes)
    
    def __repr__ (self): 
        print (self.graph_dict)


genome_graph = Graph()

# Note:  nodes can be added simply via connect, the connect function will take care of new vs existing nodes
for line in graph_adj_list:  #🔴 Can be cleaned up to replace graph_adj_list
    genome_graph.connect(line[0], line[1], line[2])

# Verification 
for node in genome_graph.nodes(): 
    if node < 10: 
        print (f'{node} : {genome_graph.get(node)}')



0 : {302: -5, 224: -16}
1 : {511: -5, 344: -6, 228: -18}
2 : {223: -22}
3 : {548: -5, 355: -24}
4 : {436: -17}
5 : {102: -5, 156: -5, 185: -15}
6 : {22: -7, 485: -7, 433: -12}
7 : {231: -25}
8 : {256: -6, 590: -23}
9 : {43: -20}


# Problem Class to solve genomic sequencing 

Similar to TSP

In [79]:
# Implement subclass of Problem to solve the genome sequencing. 
# First attempt: using Hill Climbing 
import random

class TSP_problem(Problem):

    '''
    Source: obsolete_search4e.ipynb #🔴issue with animation 
    subclass of Problem to define various functions 
    '''

    def two_opt(self, state):
        '''
        Neighbour generating function for Traveling Salesman Problem
        '''
        state2 = state[:]
        l = random.randint(0, len(state2) - 1)
        r = random.randint(0, len(state2) - 1)
        if l > r:
            l, r = r,l
        state2[l : r + 1] = reversed(state2[l : r + 1])
        return state2

    def actions(self, state):
        '''
        action that can be excuted in given state
        '''
        return [self.two_opt]
    
    def result(self, state, action):
        '''
        result after applying the given action on the given state
        '''
        return action(state)

    def path_cost(self, c, state1, action, state2):
        '''
        total distance for the Traveling Salesman to be covered if in state2
        '''
        cost = 0
        for i in range(len(state2) - 1):
            cost += distances[state2[i]][state2[i + 1]]
        cost += distances[state2[0]][state2[-1]]
        return cost
 
    def value(self, state):
        '''
        value of path cost given negative for the given state
        '''
        return -1 * self.path_cost(None, None, None, state)


initial = genome_graph.nodes() #this should be a path through the genomic reads. Expressed as a list. e.g. [1,3,5,12,32,...,23]. 
tsp_problem = TSP_problem(initial)

print (tsp)

TSP_problem([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 21

In [85]:
# Simulated Annealing stuff 

def hill_climbing(problem):
    """
    [Figure 4.2]
    From the initial node, keep choosing the neighbor with highest value,
    stopping when no neighbor is better.
    """
    current = Node(problem.initial)
    while True:
        neighbors = current.expand(problem)
        if not neighbors:
            break
        neighbor = argmax_random_tie(neighbors, key=lambda node: problem.value(node.state))
        if problem.value(neighbor.state) <= problem.value(current.state):
            break
        current = neighbor
    return current.state


def exp_schedule(k=20, lam=0.005, limit=100):
    """One possible schedule function for simulated annealing"""
    return lambda t: (k * 2.718281**(-lam * t) if t < limit else 0)


def simulated_annealing(problem, schedule=exp_schedule()):
    """[Figure 4.5] CAUTION: This differs from the pseudocode as it
    returns a state instead of a Node."""
    current = Node(problem.initial)
    for t in range(sys.maxsize):
        T = schedule(t)
        if T == 0:
            return current.state
        neighbors = current.expand(problem)
        if not neighbors:
            return current.state
        next_choice = random.choice(neighbors)
        delta_e = problem.value(next_choice.state) - problem.value(current.state)
        if delta_e > 0 or probability(2.718281**(delta_e / T)):
            current = next_choice
def simulated_annealing_full(problem, schedule=exp_schedule()):
    """ This version returns all the states encountered in reaching 
    the goal state."""
    states = []
    current = Node(problem.initial)
    for t in range(sys.maxsize):
        states.append(current.state)
        T = schedule(t)
        if T == 0:
            return states
        neighbors = current.expand(problem)
        if not neighbors:
            return current.state
        next_choice = random.choice(neighbors)
        delta_e = problem.value(next_choice.state) - problem.value(current.state)
        if delta_e > 0 or probability(2.718281**(delta_e / T)):
            current = next_choice


states = simulated_annealing_full(tsp_problem)

AttributeError: 'TSP_problem' object has no attribute 'state'